In [1]:
from google.colab import drive, output
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from google.colab import drive
import pandas as pd
import time
import json
import re
import torch
import torch._dynamo
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
from huggingface_hub import login

In [ ]:
path_raw_data = '/content/drive/MyDrive/Licenta1/Licenta/products_table.csv'
df_raw = pd.read_csv(path_raw_data)

print(f"Numărul de produse: {len(df_raw)}")
print(df_raw.head())

Numărul de produse: 62646
                                         nume_produs  id_brand nume_brand
0  Sapca Unisex Baseball BMW M Motorsport Neagra ...    7375.0        BMW
1  Prosop Audi, Albastru inchis si deschis, bumba...   21197.0       Audi
2  Tricou Personalizat AudiLove go hard, ADLER, n...   11465.0      ADLER
3          Tricou VR46 MONZA RALLY REPLICA Negru 2XL  105708.0      VR|46
4              Tricou VR46 THE DOCTOR COMIC Galben L  105708.0      VR|46


In [ ]:
#approved_brands = df_raw["nume_brand"].dropna().astype(str).unique().tolist()
#approved_brands_50 = approved_brands[:50]

#print(f"Numărul de branduri aprobate: {len(approved_brands)}")
#print(approved_brands[:10])

In [ ]:
few_shot_examples = [
    {"nume_produs": "Sapca Unisex Baseball BMW M Motorsport Neagra Marime M/L", "brand": "BMW"},
    {"nume_produs": "Prosop Audi, Albastru inchis si deschis, bumbac, 80 x 150 cm", "brand": "Audi"},
    {"nume_produs": "Tricou Personalizat AudiLove go hard, ADLER, negru, XL", "brand": "ADLER"},
    {"nume_produs": "Tricou VR46 MONZA RALLY REPLICA Negru 2XL", "brand": "VR|46"},
    {"nume_produs": "Breloc Volkswagen Tiguan Metalic, Piele, Argintiu YLKRS AutoMotive", "brand": "YLKRS AUTOMOTIVE"},
    {"nume_produs": "Breloc / husa protectie cheie AUDI, cu fermoar, 8.5cm L / 5 cm l, Maro Ubitec ®", "brand": "Ubitec"},
]

#Parametrii utilizati
def build_chat_messages(nume_produs, branduri_batch, examples):

    example_str = "\n".join([f"'{ex['nume_produs']}' → Brand: {ex['brand']}" for ex in examples])

    user_prompt = f"""
FEW-SHOT EXAMPLES:
{example_str}

### NEW INPUT ###
Product title: '{nume_produs}'
Choose the correct brand only from the following approved list: {branduri_batch}
Rules:
- Select ONLY from the list above.
- If multiple brands are mentioned, the correct brand is NEVER in the middle.
- Return only the brand name, exactly as it appears in the list.
- Return the result strictly in the following JSON format:
{{"brand": "<brand_name>"}}
- Replace <brand_name> with the correct brand from the list above.
"""
    return [
        {"role": "system", "content": [{"type": "text", "text": "You are an emag assistant that extracts brands in JSON format."}]},
        {"role": "user", "content": [{"type": "text", "text": user_prompt}]},
    ]


In [ ]:
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
from huggingface_hub import login
login("YOUR_HF_TOKEN")
model_id = "google/gemma-3-4b-it"
model = Gemma3ForConditionalGeneration.from_pretrained(model_id, device_map="auto").eval()
processor = AutoProcessor.from_pretrained(model_id)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [ ]:

# Configurare PyTorch Dynamo

torch._dynamo.config.cache_size_limit = 64
torch._dynamo.config.suppress_errors = True


def parse_json_from_response(response_text):
    matches = re.findall(r"\{.*?\}", response_text)
    for match in reversed(matches):
        try:
            data = json.loads(match)
            if "brand" in data:
                return data.get("brand", "")
        except json.JSONDecodeError:
            continue
    return ""



results = []
batch_size = 100
nr_produse = min(len(df_raw), 1000)
output_path = "/content/drive/MyDrive/Licenta1/Licenta/Rezultate_gemma_f.csv"
retry_wait_time = 1
max_retries = 5


for start in range(0, nr_produse, batch_size):
    end = start + batch_size
    df_batch = df_raw.iloc[start:end].copy()
    branduri_batch = df_batch["nume_brand"].dropna().astype(str).unique().tolist()

    print(f" Rulez batch {start} → {end-1} (cu {len(branduri_batch)} branduri)")


    for index, row in df_batch.iterrows():
        nume_produs = row['nume_produs']
        brand_corect = row['nume_brand']
        brand_extras = ""
        numar_erori = 0
        timp_executie = ""  # devine string gol daca nu reuseste

        messages = build_chat_messages(nume_produs, branduri_batch, few_shot_examples)

        for attempt in range(max_retries):
            try:
                inputs = processor.apply_chat_template(
                    messages,
                    add_generation_prompt=True,
                    tokenize=True,
                    return_dict=True,
                    return_tensors="pt"
                ).to(model.device, dtype=torch.bfloat16)

                input_len = inputs["input_ids"].shape[-1]
                start_t = time.time()

                with torch.inference_mode():
                    generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
                    generation = generation[0][input_len:]

                decoded = processor.decode(generation, skip_special_tokens=True)
                durata = time.time() - start_t

                brand_extras = parse_json_from_response(decoded)

                if brand_extras:
                    timp_executie = f"{durata:.3f}"
                    break

            except Exception as e:
                print(f" Eroare la index {index}, încercarea {attempt+1}: {e}")
                brand_extras = ""

            numar_erori += 1
            if attempt < max_retries - 1:
                time.sleep(retry_wait_time)

        brand_extras_status = (
            "Corect" if brand_extras.strip().lower() == brand_corect.strip().lower() else "Incorect"
        ) if brand_extras else "Incorect"



        results.append({
            "index": index,
            "model": "gemma-3-4b-it",
            "nume_produs": nume_produs,
            "brand_corect": brand_corect,
            "brand_extras": brand_extras,
            "brand_extras_status": brand_extras_status,
            "timp_executie_secunde": timp_executie,
            "numar_erori": numar_erori
        })


    df_partial = pd.DataFrame(results)
    df_partial.to_csv(output_path, index=False)
    print(f" Salvare intermediara: {end} produse procesate in:\n{output_path}")


print(f"\n Procesare completa! {len(results)} produse salvate in:\n{output_path}")



The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 Rulez batch 0 → 99 (cu 24 branduri)


/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are n

 Salvare intermediară: 100 produse procesate în:
/content/drive/MyDrive/Licenta1/Licenta/Rezultate_gemma_f.csv
 Rulez batch 100 → 199 (cu 15 branduri)


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not va

...

 Salvare intermediară: 800 produse procesate în:
/content/drive/MyDrive/Licenta1/Licenta/Rezultate_gemma_f.csv
 Rulez batch 800 → 899 (cu 52 branduri)


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not va

 Salvare intermediară: 900 produse procesate în:
/content/drive/MyDrive/Licenta1/Licenta/Rezultate_gemma_f.csv
 Rulez batch 900 → 999 (cu 17 branduri)


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not va

 Salvare intermediară: 1000 produse procesate în:
/content/drive/MyDrive/Licenta1/Licenta/Rezultate_gemma_f.csv

 Procesare completă! 1000 produse salvate în:
/content/drive/MyDrive/Licenta1/Licenta/Rezultate_gemma_f.csv
